In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autotime
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import pickle
path_data = "../data"
with open(os.path.join(path_data, "ginibre_pp.dat"), "rb") as data:
    ginibre_pp = pickle.load(data, encoding="bytes")
points = ginibre_pp[0].points

## section 5.2

In [ ]:
from hypton.point_pattern import PointPattern
from hypton.spatial_windows import BallWindow
import numpy as np
R = 100  # radius of the disk containing "points"
center = [0, 0]  # center of the disk containing "points"
window = BallWindow(center, R)  # creating ball window
intensity=1 / np.pi #intensity of Ginibre
ginibre_pp = PointPattern(points, window, intensity)

In [ ]:
ginibre_pp.plot()

In [ ]:
from hypton.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp)

## section 5.3

In [ ]:
from hypton.spatial_windows import BoxWindow
L = R/np.sqrt(2) #length side of the cube of restriction
bounds = np.array([[-L/2, -L/2], [L/2, L/2]]) #bounds of the cube
window = BoxWindow(bounds)
ginibre_pp_box = ginibre_pp.restrict_to_window(window)
sf_ginibre_box = StructureFactor(ginibre_pp_box)

In [ ]:
norm_k, si = sf_ginibre_box.compute_sf_scattering_intensity(
                                    max_k=10, meshgrid_size=300)



In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) #exact sf
fig = sf_ginibre_box.plot_scattering_intensity(norm_k, si, plot_type="all",
                                            exact_sf=exact_sf_ginibre,
                                            bins=50, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) #exact sf
fig = sf_ginibre_box.plot_scattering_intensity(norm_k, si, plot_type="plot",
                                            exact_sf=exact_sf_ginibre,
                                            bins=100, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

## section 5.6

In [ ]:
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=100)
H_ginibre = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=4)
print("H_ginibre=", H_ginibre)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
index_peak = hyperuniformity_test.i_first_peak

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(0, 5, 300)
y = np.linspace(0,15, 500)
fig=plt.figure(figsize=(10,6))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(mean_norm_k, mean_sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(y, exact_sf_ginibre(y), 'g', label="exact sf")
plt.plot(mean_norm_k[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_si_poisson.pdf", bbox_inches="tight")

## section 5.4

In [ ]:
r= np.linspace(0, 30, 500)
pcf_ppp = sf_ginibre.compute_pcf(method="ppp", r=r,
                                        correction="all")
pcf_ppp

In [ ]:
exact_pcf_ginibre = lambda x : 1 - np.exp(-x**2) #exact pcf
fig = sf_ginibre.plot_pcf(pcf_ppp, exact_pcf=exact_pcf_ginibre,
                    #file_name="pcf_ppp_ginibre.pdf",
                    figsize=(10,6))

In [ ]:
pcf_fv = sf_ginibre.compute_pcf(method="fv", Kest=dict(rmax=30),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
fig = sf_ginibre.plot_pcf(pcf_fv, exact_pcf=exact_pcf_ginibre,
                    #file_name="pcf_fv_ginibre.pdf",
                    figsize=(10,6))

## section 5.5

In [ ]:
domain, pcf_fv_func = sf_ginibre.interpolate_pcf(
        r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)

In [ ]:
rmax = domain["rmax"]
norm_k = np.linspace(2,10, 1000)
norm_k, sf_Ogata = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.1,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_ginibre.norm_k_min
print("norm_k_min=", norm_k_min)

In [ ]:
fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_Ogata, error_bar=True,
                             norm_k_min=norm_k_min, bins=45,
                             exact_sf=exact_sf_ginibre,
                             #file_name="sf_ginibre_Ogata.pdf"
                                          )

In [ ]:
rmax = domain["rmax"]
norm_k = np.linspace(0.3,10, 1000)
norm_k, sf_BadChou = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func, 
                                                            method ="BaddourChouinard",
                                                            norm_k=norm_k,
                                                            rmax=rmax, 
                                                            nb_points=1000)

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_BadChou,
                                    exact_sf=exact_sf_ginibre, 
                                           #file_name="sf_baddchou.pdf",
                                          error_bar=True,
                                          bins=50)

In [ ]:
norm_k_, sf_BadChou_ = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func, 
                                                            method ="BaddourChouinard",
                                                            rmax=rmax, 
                                                            nb_points=1000)

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k_, sf_BadChou_,
                                            exact_sf=exact_sf_ginibre, 
                                           #file_name="sf_baddchou.pdf",
                                          error_bar=True,
                                          bins=50)

In [ ]:
np.min(norm_k_)

## section 6.1

In [ ]:
norm_k = np.linspace(0.5,10, 1000)
r = pcf_fv["r"]
pcf_r = exact_pcf_ginibre(r)
domain, pcf_func = sf_ginibre.interpolate_pcf(r=r, pcf_r=pcf_r, clean=True) #interpolation
rmax = domain["rmax"]
norm_k, sf_Ogata = sf_ginibre.compute_sf_hankel_quadrature(pcf_func, norm_k=norm_k,
                                                rmax=rmax,
                                                step_size=0.01,
                                                nb_points=1000) 

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_Ogata, 
                             exact_sf=exact_sf_ginibre, 
                             file_name="sf_ogata_from_exact.pdf")


In [ ]:
norm_k, sf_BaddChou = sf_ginibre.compute_sf_hankel_quadrature(pcf_func, 
                                            method="BaddourChouinard",
                                            rmax=rmax, 
                                            nb_points=800) 

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_BaddChou, exact_sf=exact_sf_ginibre, file_name="sf_baddchou_from_exact.pdf")

## section 6.3

### poisson point process

In [ ]:
with open(os.path.join(path_data, "poisson_pp.dat"), "rb") as data:
    poisson_pp = pickle.load(data, encoding="bytes")
poisson_pp = poisson_pp[0]
sf_poisson = StructureFactor(poisson_pp)

In [ ]:
norm_k, si = sf_poisson.compute_sf_scattering_intensity(max_k=10, meshgrid_size=90)

In [ ]:
L=160
bounds = np.array([[-L/2, -L/2], [L/2, L/2]]) #bounds of the cube
window_res = BoxWindow(bounds)
fig = sf_poisson.plot_scattering_intensity(norm_k, si, plot_type="all",
                                            bins=60, error_bar=True,
                                            file_name="si_poisson.pdf", window_res=window_res)

In [ ]:
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=16)
print("H_poisson=", H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(0, 15, 300)
fig=plt.figure(figsize=(10,6))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_si_poisson.pdf", bbox_inches="tight")

In [ ]:
pcf_fv = sf_poisson.compute_pcf(method="fv", Kest=dict(rmax=100),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
domain, pcf_fv_func = sf_poisson.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(2,30, 1000)
norm_k, sf_Ogata = sf_poisson.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.01,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_poisson.norm_k_min
print("norm_k_min=", norm_k_min)
fig = sf_poisson.plot_sf_hankel_quadrature(norm_k, sf_Ogata, error_bar=True,
                             norm_k_min=norm_k_min, bins=80,
                             file_name="sf_poisson_Ogata.pdf")

In [ ]:
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_Ogata)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=25)
print("H_poisson=", H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(10,6))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_sf_ogata_poisson.pdf", bbox_inches="tight")

In [ ]:
domain, pcf_fv_func = sf_poisson.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(2,30, 1000)
norm_k, sf_BadChoui = sf_poisson.compute_sf_hankel_quadrature(pcf_fv_func, method="BaddourChouinard", norm_k=norm_k,
                                                        nb_points=1000,
                                                        rmax=rmax)

fig = sf_poisson.plot_sf_hankel_quadrature(norm_k, sf_BadChoui , error_bar=True, bins=20,
                             file_name="sf_poisson_badchou.pdf")

In [ ]:
import matplotlib.pyplot as plt
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_BadChoui)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=25)
print("H_poisson=", H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(10,6))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_sf_baddour_poisson.pdf", bbox_inches="tight")

### kly point process

In [ ]:
with open(os.path.join(path_data, "kly_pp.dat"), "rb") as data:
    kly_pp = pickle.load(data, encoding="bytes")
kly_pp = kly_pp[0]
sf_kly = StructureFactor(kly_pp)
norm_k, si = sf_kly.compute_sf_scattering_intensity(max_k=20, meshgrid_size=90)

In [ ]:
L=120
bounds = np.array([[0, 0], [L, L]]) #bounds of the cube
window_res = BoxWindow(bounds)
fig = sf_kly.plot_scattering_intensity(norm_k, si, plot_type="all",
                                      bins=60, error_bar=True,
                                      file_name="si_kly.pdf",
                                      window_res=window_res)

In [ ]:
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=40)
H_kly = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=2)
print("H_kly=", H_kly)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
x = np.linspace(0, 15, 300)
fig=plt.figure(figsize=(10,6))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_si_kly.pdf", bbox_inches="tight")

In [ ]:
pcf_fv = sf_kly.compute_pcf(method="fv", Kest=dict(rmax=100),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
domain, pcf_fv_func = sf_kly.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(2,30, 1000)
norm_k, sf_Ogata = sf_kly.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.05,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_kly.norm_k_min
print("norm_k_min=", norm_k_min)
fig = sf_kly.plot_sf_hankel_quadrature(norm_k, sf_Ogata, error_bar=True,
                             norm_k_min=norm_k_min, bins=80,
                             file_name="sf_poisson_Ogata.pdf")

In [ ]:
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_Ogata)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=5)
print("H_poisson=", H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(10,6))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_sf_ogata_poisson.pdf", bbox_inches="tight")

In [ ]:
domain, pcf_fv_func = sf_poisson.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(2,30, 1000)
norm_k, sf_BadChoui = sf_kly.compute_sf_hankel_quadrature(pcf_fv_func, method="BaddourChouinard", norm_k=norm_k,
                                                        nb_points=1000,
                                                        rmax=rmax)

fig = sf_kly.plot_sf_hankel_quadrature(norm_k, sf_BadChoui,
                             file_name="sf_kly_badchou.pdf")

In [ ]:
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_BadChoui)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=100)
H_kly = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=4)
print("H_kly=", H_kly)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(10,6))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_sf_ogata_poisson.pdf", bbox_inches="tight")